In [15]:
import os

import discoursegraphs as dg
import pocores as pc

MATE_PARSED_TUEBA_DIR = os.path.expanduser('~/corpora/tueba/tueba_mate_output')
TUEBA_TEST_FILE = os.path.join(MATE_PARSED_TUEBA_DIR, 'text_104.bp')

In [16]:
pocograph = pc.run_pocores(TUEBA_TEST_FILE, input_format='2009',
                           output_dest='/tmp/{}.pocores'.format(os.path.basename(TUEBA_TEST_FILE)))

In [12]:
dg.write_conll(pocograph.document, '/tmp/{}.pocores.conll'.format(pocograph.document.name),
               markable_layer='pocores:markable')

In [5]:
TUEBADZ8_FILE = os.path.expanduser(
    '~/corpora/tueba/TuebaDZ8.0/tuebadz-8.0-mit-NE+Anaphern+Diskurs.exml.xml')

In [7]:
tueba_corpus = dg.read_exportxml(TUEBADZ8_FILE)

In [8]:
for doc in tueba_corpus:
    dg.write_conll(doc, '/tmp/{}.tueba.conll'.format(doc.name))

/usr/local/lib/python2.7/dist-packages/discoursegraphs-0.1.2-py2.7.egg/discoursegraphs/discoursegraph.py:807: UserWarning: Can't reliably extract span 's504_509' from cyclical graph'text_29'.Maximum recursion depth may be exceeded.
  docgraph))
/usr/local/lib/python2.7/dist-packages/discoursegraphs-0.1.2-py2.7.egg/discoursegraphs/discoursegraph.py:807: UserWarning: Can't reliably extract span 's501_3' from cyclical graph'text_29'.Maximum recursion depth may be exceeded.
  docgraph))
/usr/local/lib/python2.7/dist-packages/discoursegraphs-0.1.2-py2.7.egg/discoursegraphs/discoursegraph.py:807: UserWarning: Can't reliably extract span 's510_503' from cyclical graph'text_29'.Maximum recursion depth may be exceeded.
  docgraph))
/usr/local/lib/python2.7/dist-packages/discoursegraphs-0.1.2-py2.7.egg/discoursegraphs/discoursegraph.py:807: UserWarning: Can't reliably extract span 's507_501' from cyclical graph'text_29'.Maximum recursion depth may be exceeded.
  docgraph))
/usr/local/lib/python2

RuntimeError: maximum recursion depth exceeded